In [2]:
import numpy as np


def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                                [w[2], 0, -w[0]],
                                [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

def BodyTwist(V_bmat):
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    w_b = w_b.reshape(-1,1)
    v_b = V_bmat[:3, 3]
    v_b = v_b.reshape(-1,1)
    output = np.vstack((w_b, v_b))
    return output
    

In [3]:
import numpy as np
import time

from scipy.linalg import logm

q = np.array([0, 0.166, -0.01])

M = np.vstack([np.hstack([np.eye(3), q.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])

w1 = np.array([0, 0, 1])
w2 = np.array([1, 0, 0])
w3 = np.array([1, 0, 0])
w4 = np.array([1, 0, 0])
w5 = np.array([0, 0, 1])

q1 = np.array([0, 0, 0])
q2 = np.array([0, 0, 0.073])
q3 = np.array([0, 0.083, 0.073])
q4 = np.array([0, 0.166, 0.073])
q5 = np.array([0, 0.166, 0.033])

v1 = -np.cross(w1, q1)
v2 = -np.cross(w2, q2)
v3 = -np.cross(w3, q3)
v4 = -np.cross(w4, q4)
v5 = -np.cross(w5, q5)

s1 = np.array([w1, v1])
s2 = np.array([w2, v2])
s3 = np.array([w3, v3])
s4 = np.array([w4, v4])
s5 = np.array([w5, v5])

w1 = w1.reshape(-1,1)
w2 = w2.reshape(-1,1)
w3 = w3.reshape(-1,1)
w4 = w4.reshape(-1,1)
w5 = w5.reshape(-1,1)

v1 = v1.reshape(-1,1)
v2 = v2.reshape(-1,1)
v3 = v3.reshape(-1,1)
v4 = v4.reshape(-1,1)
v5 = v5.reshape(-1,1)

s1 = s1.reshape(-1,1)
s2 = s2.reshape(-1,1)
s3 = s3.reshape(-1,1)
s4 = s4.reshape(-1,1)
s5 = s5.reshape(-1,1)

# start position : [0.15, 0.07, 0.0701]
x_d = np.array([0.02, 0.12, 0.05]) 
x_d = x_d.reshape(-1,1)
T_sd = np.vstack([np.hstack([np.eye(3), x_d]),
                  np.array([[0, 0, 0, 1]])])

th = np.deg2rad([-86.9474, 41.3064, -44.3338, 3.0274, 86.9474])
# th = np.deg2rad([-65.28392204, 47.79348963, -55.77497405, 7.98148442, 65.28392204])
# th = np.deg2rad([-81.2715, 53.2575, -43.9620, -9.2955, 81.2715])
th = th.reshape(-1,1)
        
w_b = np.array([1, 1, 1])
w_b = w_b.reshape(-1,1)
v_b = np.array([1, 1, 1])
v_b = v_b.reshape(-1,1)

k = 1
dt = 1

while (np.linalg.norm(w_b) > 0.001 or np.linalg.norm(v_b) > 0.0001):
    # Forward Kinematic
    exp1 = screw_exp(s1, th[0])
    exp2 = screw_exp(s2, th[1])
    exp3 = screw_exp(s3, th[2])
    exp4 = screw_exp(s4, th[3])
    exp5 = screw_exp(s5, th[4])
    T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
    T_sb = T_sb.astype(dtype = 'float64')

     # Inverse Kinematic
    T_bs = np.linalg.inv(T_sb)
    AdT_bs = Ad(T_bs)

    Js1 = s1
    Js2 = Ad(exp1) @ s2
    Js3 = Ad(exp1 @ exp2) @ s3
    Js4 = Ad(exp1 @ exp2 @ exp3) @ s4
    Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s5
    Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
    Js = Js.astype(dtype = 'float64')
    Jb = AdT_bs @ Js
    Jb = Jb.astype(dtype = 'float64')

    T_bd = T_bs @ T_sd
    V_bmat = logm(T_bd) * 0.07
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    v_b = V_bmat[:3, 3]
    V_b = np.hstack([w_b, v_b])

    th_center = np.zeros((1, 5))
    th_center[0,2] = -np.pi / 3
    th_center[0,3] = np.pi / 6
    grad_H = np.zeros((1, 5))
    grad_H[0,2] = th[2] - th_center[0,2]
    grad_H[0,3] = th[3] - th_center[0,3]
    grad_H = grad_H.reshape(-1,1)
    gamma = -0.2

    Jvb = np.linalg.pinv(Jb) @ V_b
    Jvb = Jvb.reshape(-1,1)
    dth = Jvb + gamma * (np.eye(5) - np.linalg.pinv(Jb) @ Jb) @ grad_H

    th = th + dth*dt
    th_deg = np.rad2deg(th)

    thd = np.array([th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90])

    k = k + 1

    if k == 1:
        break


In [131]:
thd

array([[80.04383307],
       [61.9950477 ],
       [18.73233924],
       [ 9.27261305],
       [80.04383307]])